### 경기도 남부 경찰청
- 산하 경찰서를 크롤링하여 지도에 표시하기

In [1]:
import requests, folium
import pandas as pd
from urllib.parse import quote
from bs4 import BeautifulSoup

- Step 1, 2

In [2]:
url = 'https://www.ggpolice.go.kr/main/html.do?menu=HC44'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
lis = soup.select('li.road')
len(lis)

31

- Step 3.

In [4]:
li = lis[0]
name = li.find('a').string.strip()
href = li.find('a')['href']
name, href

('수원중부서', 'https://www.ggpolice.go.kr/swjb/html.do?menu=HC24')

In [5]:
sub_res = requests.get(href)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [6]:
sub_lis = sub_soup.select('.basic_list7 > li')
len(sub_lis)

3

In [7]:
name = sub_lis[0].get_text().strip()[6:]
name

'수원중부경찰서'

In [8]:
addr = sub_lis[1].get_text().strip()[9:-9]
addr

'경기도 수원시 장안구 정자천로 199'

- Step 4

In [9]:
lines = []
for li in lis:
    href = li.find('a')['href']
    sub_res = requests.get(href)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis = sub_soup.select('.basic_list7 > li')
    name = sub_lis[0].get_text().strip()[6:]
    addr = sub_lis[1].get_text().strip()[9:-9]
    lines.append({'name':name, 'addr':addr})

IndexError: list index out of range

- Step 5. 오류 해결
    - 어디서 발생했는지 파악
    - 파악한 곳으로 가서 해결

In [10]:
# 어디서 에러가 발생했는지 파악
lines = []
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        sub_res = requests.get(href)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis = sub_soup.select('.basic_list7 > li')
        name = sub_lis[0].get_text().strip()[6:]
        addr = sub_lis[1].get_text().strip()[9:-9]
        lines.append({'name':name, 'addr':addr})
    except:
        print(index)

6
25


In [11]:
# 에러를 찾아서 해결
# index=6, 군포경찰서로 찾아가서 원인 파악
li = lis[6]
href = li.find('a')['href']
print(href)
sub_res = requests.get(href)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
sub_lis = sub_soup.select('.basic_list7 > li')
print(len(sub_lis))

https://www.ggpolice.go.kr/gunpo/html.do?menu=HC24
0


In [12]:
lines = []
for li in lis:
    href = li.find('a')['href']
    sub_res = requests.get(href)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis = sub_soup.select('.basic_list7 > li')
    if len(sub_lis) == 0:
        continue
    name = sub_lis[0].get_text().strip()[6:]
    addr = sub_lis[1].get_text().strip()[9:-9]
    lines.append({'name':name, 'addr':addr})

In [13]:
# 정확한 원인을 찾지 못하는 경우, just skip
lines = []
for li in lis:
    try:
        href = li.find('a')['href']
        sub_res = requests.get(href)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis = sub_soup.select('.basic_list7 > li')
        name = sub_lis[0].get_text().strip()[6:]
        addr = sub_lis[1].get_text().strip()[9:-9]
        lines.append({'name':name, 'addr':addr})
    except:
        continue

- Step 7

In [14]:
df = pd.DataFrame(lines)
print(df.shape)
df

(29, 2)


,name,addr
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199
1,수원남부경찰서,경기 수원시 영통구 매봉로 52
2,수원서부경찰서,경기 수원시 권선구 서부로 1673
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63
5,과천경찰서,경기 과천시 통영로 20
6,성남수정경찰서,경기 성남시 수정구 성남대로 1259
7,성남중원경찰서,경기 성남시 중원구 금빛로2번길 10
8,분당경찰서,경기 성남시 분당구 정자일로 165
9,부천소사경찰서,경기 부천시 부광로 214


In [15]:
# 중복 여부 확인
len(df.name.unique())

28

In [16]:
# 중복 제거
df.drop_duplicates(subset='name', inplace=True)
df.shape

(28, 2)

##### 위도, 경도 정보 얻기

In [18]:
with open('../04. 지도시각화/data/kakaoapikey.txt') as f:
    kakao_key =f.read()

In [19]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}
lat_list, lng_list = [], []
for i in df.index:
    url = f'{base_url}?query={quote(df.addr[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [20]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head()

,name,addr,위도,경도
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199,37.297932,126.996317
1,수원남부경찰서,경기 수원시 영통구 매봉로 52,37.272104,127.054492
2,수원서부경찰서,경기 수원시 권선구 서부로 1673,37.258740,126.972264
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43,37.391209,126.948866
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63,37.387334,126.925974


##### 지도에 표시하기

In [21]:
police = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=10)

for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.addr[i], max_width=200),
        tooltip=df.name[i]
    ).add_to(police)

title_html = '<h3 align="center" style="font-size:20px">경기남부경찰청 산하 경찰서</h3>'
police.get_root().html.add_child(folium.Element(title_html))
police